In [1]:
import numpy as np


class SVM:

    def __init__(self, C = 1.0):
        # C = error term
        self.C = C
        self.w = 0
        self.b = 0

    # Hinge Loss Function / Calculation
    def hingeloss(self, w, b, x, y):
        # Regularizer term
        reg = 0.5 * (w * w)

        for i in range(x.shape[0]):
            # Optimization term
            opt_term = y[i] * ((np.dot(w, x[i])) + b)

            # calculating loss
            loss = reg + self.C * max(0, 1-opt_term)
        return loss[0][0]

    def fit(self, X, Y, batch_size=100000, learning_rate=0.0001, epochs=1000):
        # The number of features in X
        number_of_features = X.shape[1]

        # The number of Samples in X
        number_of_samples = X.shape[0]

        c = self.C

        # Creating ids from 0 to number_of_samples - 1
        ids = np.arange(number_of_samples)

        # Shuffling the samples randomly
        np.random.shuffle(ids)

        # creating an array of zeros
        w = np.zeros((1, number_of_features))
        b = 0
        losses = []

        # Gradient Descent logic
        for i in range(epochs):
            # Calculating the Hinge Loss
            l = self.hingeloss(w, b, X, Y)

            # Appending all losses 
            losses.append(l)
            
            # Starting from 0 to the number of samples with batch_size as interval
            for batch_initial in range(0, number_of_samples, batch_size):
                gradw = 0
                gradb = 0

                for j in range(batch_initial, batch_initial+ batch_size):
                    if j < number_of_samples:
                        x = ids[j]
                        ti = Y[x] * (np.dot(w, X[x].T) + b)

                        if ti > 1:
                            gradw += 0
                            gradb += 0
                        else:
                            # Calculating the gradients

                            #w.r.t w 
                            gradw += c * Y[x] * X[x]
                            # w.r.t b
                            gradb += c * Y[x]

                # Updating weights and bias
                w = w - learning_rate * w + learning_rate * gradw
                b = b + learning_rate * gradb
        
        self.w = w
        self.b = b

        return self.w, self.b, losses

    def predict(self, X):
        prediction = np.dot(X, self.w[0]) + self.b # w.x + b
        return np.sign(prediction)

In [2]:
def float_to_int_delta(value, byte):

    import copy

    max_value = 50
    v = copy.deepcopy(value)
    delta = max_value/(pow(2,8*byte)-1)
    converted_int = int(v/delta)

    if(converted_int >= (pow(2,8*byte)/2)-1): 
        return int(pow(2,8*byte)/2)-1
    
    elif(converted_int <= -(pow(2,8*byte)/2)+1): 
        return -int(pow(2,8*byte)/2)+1
    
    else: 
        return converted_int

def int_to_float_delta(value, byte): 
    import copy
    max_value = 50
    v = copy.deepcopy(value)
    delta = max_value/(pow(2,8*byte)-1)
    return v*delta

In [3]:
def int_to_svm_model_name(value): 
    first_value = int((value - value%16)/16)
    second_value = value%16
    return str(first_value), str(second_value) 

In [4]:
int_to_svm_model_name(41)

('2', '9')

In [5]:
def string_int_to_hex_int(str_hex): 
    return 16*int(str_hex[0]) + int(str_hex[1]) 

In [6]:
def linear_train_int_bin(file_path, class_1, class_2, b, w): 
        #svm_model format: [class1, class2, b, w_i[]]

    #Modus: type of svm model. Linear, rbf, sparse etc.

    import struct 
    
    svm_model_name = ""

    c1 = str(class_1) 
    c2 = str(class_2)

    if(len(c1) == 1): 
        c1 = "0" + c1

    if(len(c2) == 1): 
        c2 = "0" + c2

    svm_model_name = "lsm" + c1 + c2

    file = open(file_path + svm_model_name, "wb")    
    
    buf = struct.pack("BB", class_1, class_2)

    file.write(buf)

    buf = struct.pack('h', b)

    file.write(buf)

    w_size = str(len(w)) + "h"

    buf = struct.pack(w_size, *w)
    
    file.write(buf)

    file.close()

    return 0

In [7]:
def bdt_config_bin(file_path, bdt_config_aray, config_num):

    import struct 

    bca = np.array(bdt_config_aray).flatten()

    bdt_config_name = "bdtc" + str(config_num)
    
    file = open(file_path + bdt_config_name, "wb")

    for i in range(len(bca)):
        buf = struct.pack(">B", bca[i]) 
        file.write(buf)

    file.close()

In [8]:
def hsi_binary(file_path, filename, X):

    import struct

    pH = np.array(X).flatten()
        
    file = open(file_path + filename, "wb")

    for i in range(len(pH)):
        buf = struct.pack("h", float_to_int_delta(pH[i],2)) 
        file.write(buf)

    file.close()


In [10]:
def models_names(bdt, model_names): 
    
    svm_model_list = []

    for i in range(len(bdt)): 
        c1, c2 = int_to_svm_model_name(bdt[i][0])
        
        if(len(c1) == 1): 
            c1 = "0" + c1

        if(len(c2) == 1): 
            c2 = "0" + c2

        svm_model_list.append(str(model_names) + str(c1) + str(c2)) 

    return svm_model_list

In [11]:
def linear_svm_models_load(path, bdt): 

    import os 
    import struct

    svm_models_array = []

    svm_default_name = "lsm"

    svm_model_list = models_names(bdt, svm_default_name)

    for i in range(len(svm_model_list)): 
        file = open(path + svm_model_list[i], "rb")

        unpack_opt = "2b" + str(int(os.stat(path + svm_model_list[i]).st_size/2 - 1))  + "h"
        content = struct.unpack(unpack_opt,file.read())

        svm_model = []
        svm_model.append(content[0])
        svm_model.append(content[1])

        for j in range(2,len(content)): 
            svm_model.append(int_to_float_delta(content[j],2))
        
        svm_models_array.append(svm_model)

        file.close()

    return svm_models_array  

In [12]:
def rotate_image(image): 
    rotated_image = []
    for i in range(len(image[0])): 
        x_array = []
        for j in range(len(image)-1,-1,-1):
            x_array.append(image[j][i])
        rotated_image.append(x_array)

    return np.array(rotated_image)

In [13]:
def svm_branch_models_to_bin(svm_branch_model, b_extra):

    sbm = copy.deepcopy(svm_branch_model)
    class_1 = sbm.classes_[0]
    class_2 = sbm.classes_[1]
    b = float_to_int_delta(sbm.intercept_[0] + b_extra ,2)
    w = sbm.coef_[0]
    tw = []

    for i in range(len(w)): 
        tw.append(float_to_int_delta(w[i],2))

    linear_train_int_bin("svm_model_files/", class_1, class_2, b, tw)

In [14]:
def bdt_string_to_config_format(bdt):

    import copy

    config_bdt = []

    for i in range(len(bdt)): 
        bdt_index = []
        bdt_i = copy.deepcopy(bdt[i][0]) 

        if(bdt_i[2] == "0"): 
            bdt_i = bdt_i[0] + bdt_i[1] + bdt_i[3] 

        if(bdt_i[0] == "0"): 
            bdt_i = bdt_i[1] + bdt_i[2]

        bdt_index.append(string_int_to_hex_int(bdt_i))
        bdt_index.append(int(bdt[i][1]))
        bdt_index.append(int(bdt[i][2]))
        config_bdt.append(bdt_index)

    return config_bdt

bdt = bdt_string_to_config_format([["0203", "1", "2"]])

print(bdt)

#linear_svm_models_load("svm_model_files", bdt)

[[35, 1, 2]]


In [15]:
def svm_linear(X_i, svm_model):
    pred = 0 

    for i in range(len(X_i)): 
        pred += (X_i[i] * svm_model[3+i]) + svm_model[2]
        #pred = np.dot(X_i, np.array(svm_model[3:len(X_i)+3]).T) + svm_model[2]
    if(pred >= 0): 
        return -1
    else:
        return 1

In [16]:
def bdt_execute(X, bdt): 

    # bdt : [svm_model_classes, send_index1, send_index2]
    
    svm_models = linear_svm_models_load("svm_model_files/", bdt)
    svm_models_index = 0 
    prevoius_index = 0 
    pixel_holder = 0 
    labeled_image = [] 

    for i in range(len(X)): 

        svm_models_index = 0

        while(True):

            prevoius_index = svm_models_index

            pixel_holder = svm_linear(X[i], svm_models[svm_models_index])

            if(pixel_holder >= 0): 
                svm_models_index = int(bdt[svm_models_index][1])

            else: 
                svm_models_index = int(bdt[svm_models_index][2])

            if(svm_models_index == 0): 

                if(pixel_holder >= 0): 
                    labeled_image.append(svm_models[prevoius_index][0])
                else: 
                    labeled_image.append(svm_models[prevoius_index][1])

                break

    return labeled_image, svm_models

In [49]:
from SVMBDT import *
from HelperFunctions_ import *
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

trondheim_data = plt.imread("../Training_data/Trondheim_2022_08_23T10_26_43-bin3.png")
trondheim_gt = np.load("../Training_data/Trondheim_labels_2022-08-23.npz")["labels"]

#Preprocess the labeling 

new_trondheim_gt = []
y = trondheim_gt.flatten()

for i in range(0,len(y),3): 
    new_trondheim_gt.append(y[i])

new_trondheim_gt = np.array(new_trondheim_gt)

#Make labels into land, water and clouds 
#water  = 2
#cropland = 3
#pasture = 4
#city = 5
#rock = 6
#thin_forest = 7
#thick_forest = 8 
#cloud = 9
#thin_clouds = 10

trondheim_new_labels = copy.deepcopy(new_trondheim_gt)



#trondheim_new_labels[trondheim_new_labels == 9] = 3

nl = [4,5,6]

for i in range(len(nl)):
        trondheim_new_labels[trondheim_new_labels == nl[i]] = 3

trondheim_new_labels[trondheim_new_labels == 10] = 5


print(classesInLabels(trondheim_new_labels))

new_trondheim = rotate_image(trondheim_data)

X = np.array(new_trondheim).reshape((-1,3))
#X = preprocessing.scale(X, axis=0)
y = trondheim_new_labels.flatten()

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.9, random_state=1)


#tb = {0 : [[3,8,9],[2]], 1 : [[[3],[8,9]],[]], 2 : [[[8],[9]],[]]}
#svm_tb = {0 : LinearSVC(C = 1, class_weight="balanced", intercept_scaling=1), 
#          1 : [LinearSVC(C = 1, class_weight="balanced", intercept_scaling=1), False],
#          2 : [LinearSVC(C = 1, class_weight="balanced", intercept_scaling=1), False]}
#svm_tb = {0 : SVM(), 1 : [SVM(), False]}

#tb = {0 : [[3,7,8], [2,5,9]], 
#      1 : [[[3],[7,8]],[[2],[5,9]]], 
#      2 : [[[7],[8]],[[5],[9]]]  
#      }

#svm_tb = {0 : LinearSVC(C = 1, class_weight="balanced", intercept_scaling=1),
#          1 : [LinearSVC(C = 1, class_weight="balanced", intercept_scaling=1),LinearSVC(C = 1, class_weight="balanced", intercept_scaling=1)], 
#          2 : [LinearSVC(C = 1, class_weight="balanced", intercept_scaling=1),LinearSVC(C = 1, class_weight="balanced", intercept_scaling=1)]
#          }


tb = { 0 : [[2],[3,5,7,8,9]], 1 : [[],[[5,9],[3,7,8]]], 2 : [[],[[3],[7,8]]], 3 : [[],[[7],[8]]]
      
}


svm_tb = {0 : LinearSVC(C = 1, class_weight="balanced", intercept_scaling=1),
          1 : [False,LinearSVC(C = 1, class_weight="balanced", intercept_scaling=1)], 
          2 : [False,LinearSVC(C = 1, class_weight="balanced", intercept_scaling=1)], 
          3 : [False,LinearSVC(C = 1, class_weight="balanced", intercept_scaling=1)]
          }



svm_branch_models = SvmDesionTreeTrain(X_train, y_train, tb, svm_tb)

[2 3 5 7 8 9]
The training time is: 0.111 sec.


In [18]:
X

array([[0.32941177, 0.39607844, 0.49411765],
       [0.34901962, 0.43137255, 0.5137255 ],
       [0.35686275, 0.43529412, 0.52156866],
       ...,
       [0.14509805, 0.21176471, 0.22352941],
       [0.14509805, 0.20392157, 0.22352941],
       [0.14117648, 0.2       , 0.21960784]], dtype=float32)

In [176]:
#svm_branch_models_to_bin(svm_branch_models[0],-2.8)
#svm_branch_models_to_bin(svm_branch_models[1][0],0)
#svm_branch_models_to_bin(svm_branch_models[1][1],1.7)
#svm_branch_models_to_bin(svm_branch_models[2][0],-1.25)
#svm_branch_models_to_bin(svm_branch_models[2][1],2)

svm_branch_models_to_bin(svm_branch_models[0],-0.7)
svm_branch_models_to_bin(svm_branch_models[1][1],2.4)
svm_branch_models_to_bin(svm_branch_models[2][1],-0.1)
svm_branch_models_to_bin(svm_branch_models[3][1],0)
#svm_branch_models_to_bin(svm_branch_models[2][1],2)

In [190]:
#from tempfile import TemporaryFile
#outfile = TemporaryFile("../Z_Labeled_HYSPO_Data/penguin.npy")

test_name = "../Training_data/kampala_2023-02-04_0740Z-bin3.png"
test_data = plt.imread(test_name)

X_test = np.array(test_data).reshape((-1,3))


#s_bdt = [["0203","0","0"],["0305", "2", "4"], ["0307", "0", "3"], ["0708", "0", "0"], ["0509", "0", "0"]]
s_bdt = [["0203", "0", "0"], ["0708", "0", "0"]]



bdt = bdt_string_to_config_format(s_bdt)

pl, svm_models = bdt_execute(X_test, bdt)
pl = np.array(pl)

pl[pl == 2] = 0
pl[pl == 3] = 7

plt.imshow(pl.reshape(len(test_data),len(test_data[0])))
plt.axis('off')
plt.savefig("../Z_Labeled_HYSPO_Data/kampala", dpi = 300, bbox_inches="tight")

np.save("../Z_Labeled_HYSPO_Data/kampala", pl)

plt.close()

In [123]:
np.load("../Z_Labeled_HYSPO_Data/tharthar.npy")

array([0, 0, 0, ..., 7, 7, 7])

In [22]:
s_bdt = [["0203","0","1"]]
bdt = bdt_string_to_config_format(s_bdt)

linear_svm_models_load("svm_model_files/", bdt)

[[2,
  3,
  0.11062790875104905,
  12.092774853131914,
  22.854200045777066,
  -23.58281834134432]]

In [58]:
classesInLabels(pl)

array([2, 3])

In [23]:
pl = np.array(pl)
plt.imshow(y.reshape(956,228))
plt.axis('off')
plt.savefig("Tronhdeim_gt", dpi = 300, bbox_inches="tight")
plt.close()

In [24]:
classesInLabels(pl)

array([2, 3, 5, 7, 8, 9])

In [25]:
np.sum(pl == y)/len(y)

0.6767369522131689

In [26]:
test_name = "../Training_data/losangeles_2023-01-31_1756Z-bin3.png"
test_data = plt.imread(test_name)

X_test = np.array(test_data).reshape((-1,3))
hsi_binary("hsi_images/", "vigo2", X_test)
#X_test = preprocessing.scale(X_test, axis=0)


s_bdt = [["0203","0","1"],["0305", "2", "4"], ["0307", "0", "3"], ["0708", "0", "0"], ["0509", "0", "0"]]

bdt = bdt_string_to_config_format(s_bdt)

pl, svm_models = bdt_execute(X_test, bdt)


hypso_color_label = { 0 : [0,0,200], 1 : [255,165,0],
                      2 : [150,150,150], 3: [144, 238, 144], 4 : [34,139,34], 5 : [240, 240, 240]}


hypso_color_label = { 0 : [0,0,139], 
                      1 : [13,152,186], 
                      2 : [144,238,144], 
                      3 : [1,50,32], 
                      4 : [240,0,0], 
                      5 : [20,20,20], 
                      6 : [255,255,255], 
                      7 : [194,178,128], 
                      8 : [180,180,180], 
                      9 : [100,100,100], 
                      10 : [200,200,0]   
}


pl = np.array(pl)

t=255
cmap = {}

for i in range(len(hypso_color_label)): 
    cmap[i] = np.array(hypso_color_label[i] + [255])/255

new_y = copy.deepcopy(pl)

yout_color = newColorLabels(new_y,len(test_data),len(test_data[0]),trondheim_color_label)



#pl = np.array(pl)

#plt.imshow(pl.reshape(len(test_data), len(test_data[0])))
plt.imshow(yout_color)
plt.axis('off')
plt.savefig("../new_bdt_testing/" + test_name.replace("../Training_data/",""), dpi = 300, bbox_inches="tight")
plt.close()



In [27]:
hsi_binary("hsi_images/", "los_angeles", X_test)
len(X_test)

217485

In [194]:
def store_labeled_image(predicted_label, x_dim, y_dim, path): 

    hypso_color_label = { 0 : [0,0,139], 
                        1 : [13,152,186], 
                        2 : [144,238,144], 
                        3 : [1,50,32], 
                        4 : [240,0,0], 
                        5 : [20,20,20], 
                        6 : [255,255,255], 
                        7 : [194,178,128], 
                        8 : [180,180,180], 
                        9 : [100,100,100], 
                        10 : [200,200,0]   
    }


    #t=255
    for i in range(len(hypso_color_label)): 
        cmap[i] = np.array(hypso_color_label[i] + [255])/255

    new_predicted_label = copy.deepcopy(predicted_label)
    new_predicted_label = np.array(new_predicted_label)

    np.save(path, new_predicted_label)

    yout_color = newColorLabels(new_predicted_label,x_dim,y_dim, hypso_color_label)

    plt.imshow(yout_color)
    plt.axis('off')
    plt.savefig(path, dpi = 300, bbox_inches="tight")
    plt.close()
        

In [195]:
store_labeled_image(pl, len(test_data), len(test_data[0]), "test")

In [44]:
import scipy.io as sc
gt = sc.loadmat("../Training_data/end3.mat")
samson = sc.loadmat("../Training_data/samson.mat")['V']
labels = np.argmax(gt['A'].T.reshape((-1,3)), axis=1)

new_labels = copy.deepcopy(labels)
#new_labels[new_labels == 1] = 0  
samson = reshape_sj(copy.deepcopy(samson))

X = samson.transpose(0,2,1).reshape((-1,156))
#X = preprocessing.scale(X, axis=0)
y = new_labels.flatten()

# split samson into train test sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.5, random_state=1)

tb = {0 : [[0,1],[2]], 1 : [[[0],[1]],[]]}
svm_tb = {0 : LinearSVC(C = 1, class_weight="balanced", intercept_scaling=1), 1 : [LinearSVC(C = 1, class_weight="balanced", intercept_scaling=1), False]}
#svm_tb = {0 : SVM(), 1 : [SVM(), False]}

svm_branch_models = SvmDesionTreeTrain(X_train, y_train, tb, svm_tb)

The training time is: 0.107 sec.


In [31]:
svm_branch_models_to_bin(svm_branch_models[0],-2.373)
svm_branch_models_to_bin(svm_branch_models[1][0],-0.52)

In [32]:
def bdt_execute(X, bdt): 

    # bdt : [svm_model_classes, send_index1, send_index2]
    
    svm_models = linear_svm_models_load("svm_model_files/", bdt)
    svm_models_index = 0 
    prevoius_index = 0 
    pixel_holder = 0 
    labeled_image = [] 

    for i in range(len(X)): 

        svm_models_index = 0

        while(True):

            prevoius_index = svm_models_index

            pixel_holder = svm_linear(X[i], svm_models[svm_models_index])

            if(pixel_holder >= 0): 
                svm_models_index = int(bdt[svm_models_index][1])

            else: 
                svm_models_index = int(bdt[svm_models_index][2])

            if(svm_models_index == 0): 

                if(pixel_holder >= 0): 
                    labeled_image.append(svm_models[prevoius_index][0])
                else: 
                    labeled_image.append(svm_models[prevoius_index][1])

                break

    return labeled_image, svm_models

In [33]:
#s_bdt = [["0203", "1", "0"],["0209", "0", "0"]]
s_bdt = [["0002", "1", "0"], ["0001", "0", "0"]]
bdt = bdt_string_to_config_format(s_bdt)
pl, svm_models = bdt_execute(X, bdt)

In [34]:
pl = np.array(pl)
plt.imshow(pl.reshape(95,95))
plt.axis('off')
plt.savefig("Samson_few_labels2", dpi = 300, bbox_inches="tight")
plt.close()

In [35]:
classesInLabels(pl)

array([0, 1, 2])

In [36]:
np.sum(pl == y)/len(y)

0.9785041551246537

In [37]:
"../Onboard_classifiication/Implementation/src/../../../Ground_Training/svm_model_files/"

'../Onboard_classifiication/Implementation/src/../../../Ground_Training/svm_model_files/'

In [38]:
def analyse_labeled_satellite_images(file_path):
    import struct
    with open(file_path, mode='rb') as file: # b is important -> binary
        fileContent = file.read()
    filesize = len(fileContent)
    unpack_opt = str(filesize) + "B"
    return struct.unpack(unpack_opt, fileContent)



In [39]:
labeled_image = analyse_labeled_satellite_images("label_image_bin/los_angeles")

In [47]:
import numpy as np
classesInLabels(labeled_image)

test_name = "../Training_data/losangeles_2023-01-31_1756Z-bin3.png"
test_data = plt.imread(test_name)


labeled_image = np.array(labeled_image)
plt.imshow(labeled_image.reshape(len(test_data),len(test_data[0])))
plt.axis('off')
plt.savefig("los_angeles", dpi = 300, bbox_inches="tight")
plt.close()

In [41]:
classesInLabels(labeled_image)

array([2, 3])

In [42]:
int_to_svm_model_name(41)

('2', '9')

In [43]:
# 0 Water 
# 1 Strange Water 
# 2 Light Forest 
# 3 Dark Forest 
# 4 Urban 
# 5 Rock 
# 6 Ice 
# 7 Sand
# 8 Thick Clouds 
# 9 Thin Clouds 
# 10 Crop 